<a href="https://colab.research.google.com/github/hariskhan-hk/sales_data_2023/blob/main/Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
file = '/content/Analytics Case Study - Launchpad 2024 (Dataset).xlsx'
engine = create_engine('sqlite://', echo = False)
df = pd.read_excel(file, sheet_name = 'Sales data')
df.to_sql('sales_data', engine, index=False, if_exists='replace')

25000

# **Task 1. Data Exploration**

Write SQL queries for the following tasks and provide a brief summary of key findings including:

● Total number of orders

● Total sales revenue

● Average order quantity

● Distribution of orders by warehouse and store

● Topselling items

In [137]:
#Total Number Of Orders

total_orders = """
SELECT COUNT(DISTINCT order_number) AS total_orders
FROM sales_data
"""

total_orders_result = pd.read_sql_query(total_orders, engine)
print("Total Number Of Orders:", total_orders_result['total_orders'][0])

Total Number Of Orders: 7495


In [138]:
#Total Sales Revenue

sales_revenue = """
SELECT ROUND(SUM((ordered_quantity * amount_per_unit) - item_discount), 3) AS sales_revenue
FROM sales_data
WHERE order_status = 'CLOSED'
"""

sales_revenue_result = pd.read_sql_query(sales_revenue, engine)
print("Total Sales Revenue:", sales_revenue_result['sales_revenue'][0])

Total Sales Revenue: 88403552.725


In [139]:
#Average Order Quantity

avg_order_quantity = """
SELECT ROUND(AVG(ordered_quantity),3) AS avg_order_quantity
FROM sales_data
WHERE order_status = 'CLOSED'"""

avg_order_quantity_result = pd.read_sql_query(avg_order_quantity, engine)
print("Average order quantity:", avg_order_quantity_result['avg_order_quantity'][0])

Average order quantity: 6.225


In [140]:
# Distribution of Orders by Warehouse and Store

dist_orders = """
SELECT store_id, order_warehouse_id, COUNT(order_number) AS distribution
FROM sales_data
GROUP BY store_id, order_warehouse_id
"""

dist_orders_result = pd.read_sql_query(dist_orders, engine)
print("Distribution of Orders by Warehouse and Store:", dist_orders_result)

Distribution of Orders by Warehouse and Store:                     store_id  order_warehouse_id  distribution
0     107AMOMq2NHOR5P0tw3KS5                11.0            19
1      10DptMbyzPdOdVyEKdYrz                 3.0             1
2     10IIYaIzgnFB5AfffemGLh                 3.0             1
3     10KvNYoKwU50tyq1jyTLmZ                11.0             2
4     10MWcgxPhNFWbnrc72JJ0k                 9.0             1
...                      ...                 ...           ...
6660   zYIQTaetDkoL3LKtxyagf                 8.0             1
6661   zYiOuc8B5E09MkIFrL7r6                 3.0             4
6662   zlSNMboC1m0qlcBItpCfX                11.0             2
6663   zsxmkTOVQCV2YADhCU3EF                 3.0             1
6664   zyOYpOejtfb6gP38fb2VH                 9.0             4

[6665 rows x 3 columns]


In [141]:
#Top Selling Items

top_selling_items = """
SELECT item_id, SUM(ordered_quantity) AS top_selling_items
FROM sales_data
GROUP BY item_id
ORDER BY ordered_quantity DESC
LIMIT 10
"""
top_selling_items_result = pd.read_sql_query(top_selling_items, engine)
print("Top Selling Items:")
print(top_selling_items_result)

Top Selling Items:
                   item_id  top_selling_items
0  P2RwFPjdKJ0oFr9OoVmrDRm             400000
1      P255207028829621388               1896
2      P431219087541607447               1595
3      P881311747325487679               5006
4      P020682599695596471               1012
5           P5532071329948                620
6           P5678188036252                516
7  P7EuMEqNrLfjaPC1nvEUqsB                500
8  P34CUAnLDL0uw3j0Hb3piTs                500
9      P693454584530622583                481


# **Task 2. Analytical Questions**

 Answer the following analytical questions using SQL queries:

 ● What is the overall discount rate (average discount per item sold)?

 ● Which warehouse has the highest average order value (total sales revenue divided by number of orders)?

 ● What is the total revenue generated by each store?

 ● Identify the top 5 customers (based on total amount spent).

 ● Calculate the month-over-month growth rate of sales revenue.

 ● Determine the percentage of orders that were canceled.

In [142]:
#Overall Discount Rate (Average Discount per Item Sold)

avg_discount_query = """
SELECT ROUND(SUM(item_discount) / SUM(ordered_quantity), 3) AS avg_discount_per_item
FROM sales_data
WHERE order_status = 'CLOSED'
"""

avg_discount_result = pd.read_sql_query(avg_discount_query, engine)
print("Overall Discount Rate:", avg_discount_result['avg_discount_per_item'][0])


Overall Discount Rate: 7.204


In [143]:
#Warehouse that has the Highest Average Order Value

highest_avg_order_value = """
SELECT order_warehouse_id, SUM(ordered_quantity * amount_per_unit - item_discount) / COUNT(order_number) AS avg_order_value
FROM sales_data
WHERE order_status = 'CLOSED'
GROUP BY order_warehouse_id
ORDER BY avg_order_value DESC
LIMIT 1
"""

highest_avg_order_value_result = pd.read_sql_query(highest_avg_order_value, engine)

print("Warehouse with the Highest Average Order Value:")
print(highest_avg_order_value_result)


Warehouse with the Highest Average Order Value:
   order_warehouse_id  avg_order_value
0                39.0          31737.5


In [144]:
#Total Revenue Generated by Each Store

total_revenue = """
SELECT store_id, SUM(ordered_quantity * amount_per_unit - item_discount) AS total_revenue
FROM sales_data
WHERE order_status = 'CLOSED'
GROUP BY store_id
ORDER BY total_revenue DESC
"""

total_revenue_result = pd.read_sql_query(total_revenue, engine)

print("Total Revenue Generated by Each Store:")
print(total_revenue_result)

Total Revenue Generated by Each Store:
                    store_id  total_revenue
0     4PdVF4u8WXhO4xnSMCYIzs      2560000.0
1     49WcD3UqaC72tOQ6tood3n      2030000.0
2     6FFhVY8Ujlw7dpqveE2UTh      1836788.0
3     6nEbvZQMYLEkxxN4JwVmEq      1819875.0
4     6p39aRrK34DT9A8RWEhSCk      1145000.0
...                      ...            ...
6219  2aT27c876xy27l3yIHwBTl           19.0
6220  161qnFBoEc2sfX09u2lDS2           19.0
6221  7K0YfPJsklVbEXSdB8kKh0           17.0
6222  2J30oJg68EqdlAuD12xAUL           17.0
6223  6JcPkFhr4qnFeFeJbgUCIv            5.0

[6224 rows x 2 columns]


In [145]:
#Top 5 Customers (Based on Total Amount Spent)

top_customers = """
SELECT order_number, SUM((ordered_quantity * amount_per_unit) - item_discount) AS total_spent
FROM sales_data
WHERE order_status = 'CLOSED'
GROUP BY order_number
ORDER BY total_spent DESC
LIMIT 5;
"""
top_customers_result = pd.read_sql_query(top_customers, engine)

print("Top 5 Customers Based on Total Amount Spent:")
print(top_customers_result)

Top 5 Customers Based on Total Amount Spent:
   order_number  total_spent
0   23046539355    2030000.0
1   23053505151    1821600.0
2   23053458040    1814400.0
3   23038626816    1280000.0
4   23038135646    1280000.0


In [146]:
#Month-over-month Growth Rate of Sales Revenue

growth_rate = """
SELECT strftime('%m-%Y', order_date) AS month_year, SUM(ordered_quantity * amount_per_unit - item_discount) AS monthly_revenue
FROM sales_data
WHERE order_status = 'CLOSED'
GROUP BY strftime('%m-%Y', order_date)
ORDER BY month_year;
"""

monthly_revenue_df = pd.read_sql_query(growth_rate, engine)

monthly_revenue_df['prev_month_revenue'] = monthly_revenue_df['monthly_revenue'].shift(1)
monthly_revenue_df['growth_rate'] = ((monthly_revenue_df['monthly_revenue'] - monthly_revenue_df['prev_month_revenue']) / monthly_revenue_df['prev_month_revenue']) * 100

print(monthly_revenue_df)

   month_year  monthly_revenue  prev_month_revenue  growth_rate
0        None     5.128246e+07                 NaN          NaN
1     01-2023     3.428816e+06        5.128246e+07   -93.313862
2     02-2023     3.767157e+06        3.428816e+06     9.867557
3     03-2023     3.499129e+06        3.767157e+06    -7.114852
4     04-2023     2.639829e+06        3.499129e+06   -24.557541
5     05-2023     9.445302e+05        2.639829e+06   -64.220023
6     06-2023     3.077181e+06        9.445302e+05   225.789593
7     07-2023     7.703175e+06        3.077181e+06   150.332215
8     08-2023     3.189381e+06        7.703175e+06   -58.596543
9     09-2023     2.792474e+06        3.189381e+06   -12.444639
10    10-2023     1.727613e+06        2.792474e+06   -38.133235
11    11-2023     3.164308e+06        1.727613e+06    83.160679
12    12-2023     1.187497e+06        3.164308e+06   -62.472153


In [147]:
# Percentage of Cancelled Orders
percentage_cancelled_orders_query = """
SELECT
    (SELECT COUNT(*) FROM sales_data WHERE order_status = 'CANCELLED') * 100.0 /
    (SELECT COUNT(*) FROM sales_data) AS Percentage_cancelled_orders;
"""

percentage_cancelled_orders_result = pd.read_sql_query(percentage_cancelled_orders_query, engine)

print("Percentage of Cancelled Orders:")
print(percentage_cancelled_orders_result['Percentage_cancelled_orders'][0])

Percentage of Cancelled Orders:
5.516
